# Math and array tools

This tutorial walks you through some common array

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/sciris/sciris/HEAD?labpath=docs%2Ftutorials%2Ftut_arrays.ipynb) to open an interactive version of this notebook.
    
</div>

## Array indexing

Let's create some data.

In [ ]:
import numpy as np

data = np.random.rand(100)
print(data)

What if we want to do something super simple, like find the indices of the values above 0.9? In Numpy, it's not super straightforward:

In [ ]:
inds = (data>0.9).nonzero()[0]
print(inds)

In Sciris, there's a function for doing exactly this:

In [ ]:
import sciris as sc

inds = sc.findinds(data>0.9)
print(inds)

Likewise, what if we want to find the value closest to, say, 0.5? In Numpy, that would be

In [ ]:
target = 0.5
nearest = np.argmin(abs(data-target))
print(nearest, data[nearest])

Which is not _too_ long, but it's a little harder to remember than the Sciris equivalent:

In [ ]:
nearest = sc.findnearest(data, target)
print(nearest, data[nearest])

The Sciris functions also work on anything "data like": for example,

In [ ]:
target = 50
data = [81, 78, 66, 25,  6,  8, 53, 96, 64, 23]

# With Numpy
ind = np.argmin(abs(np.array(data)-target))

# With Sciris
ind = sc.findnearest(data, 50)

print(ind, data[ind])

This is a simple example, but you can see how Sciris functions can do the same things with less typing.